In [1]:
# Module Importations
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Print versioning information
print(sklearn.__version__, np.__version__, pd.__version__) 

0.22.2.post1 1.19.4 1.1.4


In [6]:
# Custom Module Imports
from Source.data import load_data
from Source.data import split_data

In [3]:
# Constants

In [5]:
# Load data from pickle
original_dataset_df = load_data.load_pickled_data('full_data_df.pkl')

Loaded pickled dataframe ...


In [7]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_eval(original_dataset_df, 0.8)

In [9]:
print(training_set.info())
print(evaluation_set.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27512 entries, 129536 to 121958
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE_TIME    27512 non-null  datetime64[ns]
 1   PLANT_ID     27512 non-null  int64         
 2   SOURCE_KEY   27512 non-null  object        
 3   DC_POWER     27512 non-null  float64       
 4   AC_POWER     27512 non-null  float64       
 5   DAILY_YIELD  27512 non-null  float64       
 6   TOTAL_YIELD  27512 non-null  float64       
 7   CELL_NO      27512 non-null  object        
 8   TIME_OF_DAY  27512 non-null  object        
 9   AMB_TEMP     27512 non-null  float64       
 10  MOD_TEMP     27512 non-null  float64       
 11  IRRADIATION  27512 non-null  float64       
 12  PLANT        27512 non-null  object        
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 4.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 110044 entries